This notebook implements **solutions** presented in the [README](https://github.com/chauvinSimon/hmm_for_autonomous_driving/blob/master/README.md).

Some questions are addressed using the [hmmlearn](https://hmmlearn.readthedocs.io/en/latest/index.html#) python package.
- To install it on **Windows**, you may want to get an already [compiled version](https://www.lfd.uci.edu/~gohlke/pythonlibs/#hmmlearn).

In [39]:
!pip install hmmlearn

In [40]:
from hmmlearn import hmm
import numpy as np
from collections import Counter

In [41]:
seed = 93
np.random.seed(seed)

# Dataset
- We used **real data**.
    - The [NGSIM US-101 highway dataset](https://catalog.data.gov/dataset/next-generation-simulation-ngsim-vehicle-trajectories-and-supporting-data) contains a collection of detailed **vehicle trajectory data** recorded on southbound US 101 and Lankershim Boulevard in Los Angeles, CA, eastbound I-80 in Emeryville, CA and Peachtree Street in Atlanta, Georgia.
    
- **Dataset extraction:** 
    - We extracted 4 columns() from NGSIM US-101 highway dataset followed by location. We have taken the location **"US-101"**.
    - We have sorted our data 

In [42]:
states = ['lane_0', 'lane_1', 'lane_2', 'lane_3', 'lane_4', 'lane_5', 'lane_6', 'lane_7']
observations = ["low_speed", 'average_speed', "high_speed"]

In [43]:
discrete_model = hmm.MultinomialHMM(n_components=8,
                                    algorithm='viterbi',  # decoder algorithm.
                                    random_state=seed,
                                    n_iter=10,
                                    tol=0.01  # EM convergence threshold (gain in log-likelihood)
                                    )

# [Q1](https://github.com/chauvinSimon/hmm_for_autonomous_driving/blob/master/README.md#q1) How to easily estimate the parameters of our HMM?
Here are the results of the estimation procedure based on **counting from the labelled data**. These estimates constitute the **MLE**.

This question gives the chance to mentiont the **stationary state distribution**.
- Using **sampling**.
- Using the **transition matrix**.

In [67]:
!gdown --id "1Kr3vmpVs1G2imay6tdJAnzNQffvV69bw"
!gdown --id "1carPOnAFbl-PShBiY1O6tCgypkciA-J9"
!gdown --id "1Ke-ED02ASe8Tan8oi360EQmVGpoDIsow"
!gdown --id "1nUvFehvVRkc2eoNxvULamhUCwF3BSNzj"
!gdown --id "1nUvFehvVRkc2eoNxvULamhUCwF3BSNzj"
!gdown --id "1fxCoG9rUFJUFeCTlIoKg9zQIj_3ESimf"

Downloading...
From: https://drive.google.com/uc?id=1Kr3vmpVs1G2imay6tdJAnzNQffvV69bw
To: /content/tm.txt
100% 1.60k/1.60k [00:00<00:00, 1.39MB/s]
Downloading...
From: https://drive.google.com/uc?id=1carPOnAFbl-PShBiY1O6tCgypkciA-J9
To: /content/ini_p.txt
100% 200/200 [00:00<00:00, 298kB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ke-ED02ASe8Tan8oi360EQmVGpoDIsow
To: /content/em.txt
100% 600/600 [00:00<00:00, 339kB/s]
Downloading...
From: https://drive.google.com/uc?id=1nUvFehvVRkc2eoNxvULamhUCwF3BSNzj
To: /content/dataset_prep.ipynb
100% 14.7k/14.7k [00:00<00:00, 46.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nUvFehvVRkc2eoNxvULamhUCwF3BSNzj
To: /content/dataset_prep.ipynb
100% 14.7k/14.7k [00:00<00:00, 44.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fxCoG9rUFJUFeCTlIoKg9zQIj_3ESimf
To: /content/create_csv.ipynb
100% 1.19k/1.19k [00:00<00:00, 1.76MB/s]


In [45]:
initial_state_probability = np.loadtxt('ini_p.txt')
emission_probability = np.loadtxt('em.txt')
transition_probability = np.loadtxt('tm.txt')

In [46]:
# discrete_model.startprob_ = np.array(
#     [1/3, 2/3]
# )
# discrete_model.transmat_ = np.array([
#     [0.6, 0.4],  # P(state_t+1|state_t=state_0)
#     [0.2, 0.8]]  # P(state_t+1|state_t=state_1)
# )
# discrete_model.emissionprob_ = np.array(
#     [[0.4, 0.6],  # P(obs|state_0)
#      [0.8, 0.2]]  # P(obs|state_1)
# )
discrete_model.startprob_ = initial_state_probability
discrete_model.transmat_ = transition_probability
discrete_model.emissionprob_ = emission_probability

### Stationary distributions from sampling
Generate random samples from the HMM model, using the derived parameters.

A **large sample** is drawn.
- Counting occurences should lead to the **stationary state distributions**.


In [47]:
sample_obs, sample_states = discrete_model.sample(100000)
print(Counter(sample_states.flatten()))
print(Counter(sample_obs.flatten()))

Counter({0: 22238, 1: 21256, 2: 18587, 4: 18254, 3: 17118, 5: 1927, 6: 410, 7: 210})
Counter({1: 53427, 0: 39502, 2: 7071})


### Stationary distribution from the transition matrix
The **stationary distribution** is a **left eigenvector** (as opposed to the usual right eigenvectors) of the **transition matrix**.

In [48]:
# Compute the stationary distribution of states.
eigen_values, eigen_vectors = np.linalg.eig(discrete_model.transmat_.T)
eigen_vector = np.real_if_close(eigen_vectors[:, np.argmax(eigen_values)])

# normalisation
stationary_distribution = eigen_vector / eigen_vector.sum()
print(stationary_distribution)

[0.21959212 0.21238213 0.18631189 0.17067177 0.18530184 0.01942018
 0.00418005 0.00214002]


# [Q2](https://github.com/chauvinSimon/hmm_for_autonomous_driving/blob/master/README.md#q2) - Given a single speed observation, what is the probability for the car to be in each of the eight lanes?

Here, two methods of `hmmlearn` are used:
- The built-in [scoring](https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.MultinomialHMM.predict) method.
- The built-in [decoding](https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.MultinomialHMM.decode) method.

### Posterior probabilities
In the [README](README.md), the **posterior conditional probabilities** were computed using Bayes' rule. 

**Posterior probabilities** for each state can be computed with `predict_proba()` and corresponding most likely states with `predict()`.

In [49]:
for i in range(len(observations)):
    p = discrete_model.predict_proba(np.array([[i]]))
    most_likely = discrete_model.predict(np.array([[i]]))
    print("p(state|'{}') = {} => most likely state is '{}'".format(
        observations[i], p, states[int(most_likely[0])]))


def lane_probabilites_given_speed(speed_idx):
    return discrete_model.predict_proba(np.array([[speed_idx]]))


lane_probabilites_given_speed(2)

p(state|'low_speed') = [[0.33159203 0.23663338 0.19678764 0.10901641 0.12243604 0.00155111
  0.00198339 0.        ]] => most likely state is 'lane_0'
p(state|'average_speed') = [[0.15103211 0.19984968 0.17864305 0.21164977 0.23316107 0.01939745
  0.00494262 0.00132425]] => most likely state is 'lane_4'
p(state|'high_speed') = [[0.11619063 0.17245174 0.18619687 0.20307135 0.17202664 0.11917167
  0.01062768 0.02026343]] => most likely state is 'lane_3'


array([[0.11619063, 0.17245174, 0.18619687, 0.20307135, 0.17202664,
        0.11917167, 0.01062768, 0.02026343]])

# [Q3](https://github.com/chauvinSimon/hmm_for_autonomous_driving/blob/master/README.md#q3) - What is the probability to observe a particular sequence of speed measurements?

The presented solution uses the [scoring](https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.MultinomialHMM.predict) method of `hmmlearn`.

The **marginal probabilities of an observation sequence** can be found by:
- **The sum over any column of the product `alpha` \* `beta`** (done in th next question).
- In `hmmlearn`, this is given by the `score()` method.

In [50]:
# For a single observation
# Compute the log probability under the model.
for i in range(len(observations)):
    p = np.exp(discrete_model.score(np.array([[i]])))
    print("p({}) = {} ".format(observations[i], p))

print("---")
# Compute the log probability under the model and compute posteriors.
for i in range(len(observations)):
    p = (discrete_model.score_samples(np.array([[i]])))
    print("p({}) = {}\n   posterior p(state|{}) = {}".format(observations[i], np.exp(p[0]), observations[i], p[1]))

p(low_speed) = 0.3932705057750334 
p(average_speed) = 0.536158322304407 
p(high_speed) = 0.0705711719205599 
---
p(low_speed) = 0.3932705057750334
   posterior p(state|low_speed) = [[0.33159203 0.23663338 0.19678764 0.10901641 0.12243604 0.00155111
  0.00198339 0.        ]]
p(average_speed) = 0.536158322304407
   posterior p(state|average_speed) = [[0.15103211 0.19984968 0.17864305 0.21164977 0.23316107 0.01939745
  0.00494262 0.00132425]]
p(high_speed) = 0.0705711719205599
   posterior p(state|high_speed) = [[0.11619063 0.17245174 0.18619687 0.20307135 0.17202664 0.11917167
  0.01062768 0.02026343]]


### For the example used in [README](https://github.com/chauvinSimon/hmm_for_autonomous_driving/blob/master/README.md) : [`low_speed`, `average_speed`, `low_speed`, `low_speed`, `high_speed`,`average_speed`, `low_speed`]

In [51]:
obs_sequence = np.array([[0, 1, 0, 0, 2, 1, 0]]).T
p = np.exp(discrete_model.score(obs_sequence))
print("p({}) = {} ".format([observations[i] for i in obs_sequence.T[0]], p))

p(['low_speed', 'average_speed', 'low_speed', 'low_speed', 'high_speed', 'average_speed', 'low_speed']) = 0.0004849238753227504 


# [Q4](https://github.com/chauvinSimon/hmm_for_autonomous_driving/blob/master/README.md#q4) - Given a sequence of speed observations, what is the most likely current lane?

This question about **filtering** gives the chance to introduce the **Forward Algorithm** (and the **Backward Algorithm**).
- First, the **Forward Algorithm** is implemented to build the `alpha` table.
- Then, **filtering** can be completed by normalizing `alpha` values (using marginals).

From the `alpha` and `beta` tables, **marginal probabilities** are computed.
- For the **full observation sequence**.
- For **sub-sequences** of the observation sequence.

Finally, and as for the [README](README.md#q4), multiple inference techniques are presented.
- **Filtering**
- **Smoothing**
- **Prediction**

In [52]:
n_state = discrete_model.n_components
start_prob = discrete_model.startprob_
emit_prob = discrete_model.emissionprob_
transmat_prob = discrete_model.transmat_

### Forward algorithm

Used to build the `alpha` table for an observation sequence.

Initialization:
- `alpha`(`i`, `t=0`) = p(`obs_0`, `lane_i`) = p(`obs_0`|`lane_i`) * p(`lane_i`)

Recursion:
- `alpha`(`i`,`t+1`) = [emission at `t+1`] * SUM[over `state_j`][transition `t`->`t+1`*`alpha`(`j`,`t`)]

In [53]:
def forward_algo(obs_seq):
    alpha = np.zeros((len(obs_seq), n_state))
    alpha[0] = np.transpose(emit_prob)[obs_seq[0]] * start_prob
    for t in range(alpha.shape[0] - 1):
        alpha[t + 1] = np.transpose(emit_prob)[obs_seq[t + 1]] * np.dot(alpha[t], transmat_prob)
    return alpha

In [54]:
obs_sequence = np.array([[0, 1, 2, 2, 2, 1, 1, 0, 1, 2, 1, 0]]).T
alpha = forward_algo(obs_sequence)
print(alpha)

[[1.30405365e-01 9.30609306e-02 7.73907739e-02 4.28729406e-02
  4.81504815e-02 6.10006100e-04 7.80007800e-04 0.00000000e+00]
 [3.16736460e-02 4.22107213e-02 3.77015824e-02 4.47079078e-02
  4.92514287e-02 4.09243054e-03 1.05204209e-03 2.75343309e-04]
 [1.73285530e-03 2.56639520e-03 2.77194372e-03 3.02007073e-03
  2.55851747e-03 1.78002378e-03 1.57862275e-04 3.07243497e-04]
 [1.24323027e-04 1.80085818e-04 1.94815428e-04 2.12867625e-04
  1.80444926e-04 1.21812363e-04 1.07700876e-05 1.99678725e-05]
 [8.71526236e-06 1.26421587e-05 1.36708656e-05 1.49381453e-05
  1.26602114e-05 8.55938295e-06 7.57145715e-07 1.40592833e-06]
 [6.04092365e-06 7.81186567e-06 6.99382772e-06 8.30177355e-06
  9.14977780e-06 7.42844491e-07 1.87745869e-07 4.89847005e-08]
 [3.18555830e-06 4.20761571e-06 3.76212082e-06 4.45251052e-06
  4.90473499e-06 4.09700029e-07 1.03882057e-07 2.84107173e-08]
 [2.75034937e-06 1.95858520e-06 1.62934364e-06 9.01702640e-07
  1.01273539e-06 1.28833083e-08 1.63853669e-08 0.00000000e+00]


### Backward Algorithm
Used to build the `alpha` table for an observation sequence.
Initialization:
- `beta`(`i`, `t=T`) = `1`

Recursion:
- `beta`(`i`,`t+1`) = SUM[over `state_j`][emission at `t+1` * transition `t`->`t+1` * `beta`(`j`,`t`)]

In [55]:
def backward_algo(obs_seq):
    beta = np.zeros((len(obs_seq), n_state))
    beta[len(obs_seq) - 1] = np.ones(n_state)
    for t in reversed(range(len(obs_seq) - 1)):
        beta[t] = np.dot(beta[t + 1] * np.transpose(emit_prob)[obs_seq[t + 1]], np.transpose(transmat_prob))
    return beta

In [56]:
beta = backward_algo(obs_sequence)
print(beta)

[[1.68121943e-07 1.68135812e-07 1.68041236e-07 1.67497441e-07
  1.67764684e-07 1.66969050e-07 1.68785654e-07 1.63232782e-07]
 [3.11733587e-07 3.14915619e-07 3.13479291e-07 3.12210453e-07
  3.14085113e-07 3.02359741e-07 3.26578431e-07 2.71250567e-07]
 [4.44170024e-06 4.48697595e-06 4.46650556e-06 4.44837956e-06
  4.47510116e-06 4.30838024e-06 4.65267355e-06 3.86552887e-06]
 [6.32806287e-05 6.39382641e-05 6.36461455e-05 6.34121220e-05
  6.37954273e-05 6.12931809e-05 6.64525041e-05 5.48633239e-05]
 [9.02940930e-04 9.03094200e-04 9.02580462e-04 8.99667666e-04
  9.01114157e-04 8.96740507e-04 9.07123814e-04 8.76835806e-04]
 [1.68440579e-03 1.68481195e-03 1.68382849e-03 1.67830057e-03
  1.68108672e-03 1.67266858e-03 1.69333206e-03 1.63513428e-03]
 [3.13466534e-03 3.12798617e-03 3.13305291e-03 3.14904728e-03
  3.13864664e-03 3.18210055e-03 3.08524974e-03 3.33440315e-03]
 [7.98328814e-03 7.98393761e-03 7.97943545e-03 7.95358516e-03
  7.96629605e-03 7.92855478e-03 8.01469211e-03 7.75111854e-03]


### Marginals for observation sub-sequences
Summing over the `t`-th `alpha` column gives the probability of observing the sub-sequence `obs_sequence`[`:, t`].

- `sub_seq_marginals`[i] = p(`obs_1` ... `obs_i`) = sum over `k` of `alpha(`t`=`i`, `lane`=`k`)`

In [57]:
sub_seq_marginals = np.sum(alpha, axis=1)
print(sub_seq_marginals)

[3.93270506e-01 2.10965102e-01 1.48949120e-02 1.04508715e-03
 7.33491005e-05 3.92777434e-05 2.10545332e-05 8.28198491e-06
 4.44277183e-06 3.13676046e-07 1.67961191e-07 6.60682843e-08]


### Marginal of the full observation sequence
The marginal probability of the observation sequence can be obtained
by summing the product `alpha`*`beta` at any `t`.

In [58]:
prod = np.multiply(alpha, beta)
marginals = np.sum(prod, axis=1)
print(marginals)

[6.60682843e-08 6.60682843e-08 6.60682843e-08 6.60682843e-08
 6.60682843e-08 6.60682843e-08 6.60682843e-08 6.60682843e-08
 6.60682843e-08 6.60682843e-08 6.60682843e-08 6.60682843e-08]


### Filtering
Filtering can be obtained by normalizing the last `alpha` values (using the marginal probability of the full observation sequence).

In [59]:
print(alpha[-1] / marginals[-1])

[0.33203551 0.23652658 0.1967495  0.10887988 0.1222741  0.00155534
 0.00197909 0.        ]


### Prediction
The `pi` variable is introduced:
- `pi`(`lane i`, `time t+k+1`) = SUM over `state` `j` of [transition `j`->`i` * `pi`(`lane i`, `time t+k`)]

In [60]:
def pi_algo(k, alpha, beta):
    pi = np.zeros((k, n_state))
    pi[0] = alpha[0] / marginals[0]
    for t in range(pi.shape[0] - 1):
        pi[t + 1] = np.dot(pi[t], transmat_prob)
    return pi

In [61]:
pi = pi_algo(4, alpha, beta)
print(pi)

[[1973796.75724698 1408556.79288    1171375.56631188  648918.63115493
   728798.72745726    9232.96414847   11806.0853046        0.        ]
 [1299989.41672789 1266342.50877781 1110012.53516284 1017811.73006074
  1104985.21917146  115665.5892972    25117.1577737    12561.36753249]
 [1307153.12942345 1264157.72467731 1109001.07268439 1015909.65091114
  1103028.3983472   115615.17607764   24881.00271465   12739.36966834]
 [1307118.98615961 1264201.97139569 1109018.91873816 1015921.47620209
  1103005.91553955  115598.15998832   24881.66897446   12738.42750627]]


### Smoothing
The `gamma` variable is introduced:
- `gamma`(`k`, `t`) = p(`lane(t)`=`k` | `[observation sequence (1...T)]`)
- for `t` in [`1` ... `T`].

In [62]:
def gamma_algo(alpha, beta):
    prod = np.multiply(alpha, beta)
    marginals = np.sum(prod, axis=1)
    gamma = np.divide(prod, marginals[:, np.newaxis])
    return gamma

In [63]:
gamma = gamma_algo(alpha, beta)
print(gamma)

[[0.33183855 0.23682884 0.1968394  0.10869221 0.12226669 0.00154162
  0.0019927  0.        ]
 [0.14944749 0.20119813 0.17888561 0.21127045 0.23413868 0.0187289
  0.00520029 0.00113045]
 [0.11649801 0.17429473 0.18739554 0.20334145 0.17329986 0.11607717
  0.01111701 0.01797623]
 [0.1190774  0.17427991 0.18767327 0.20430965 0.17423733 0.11300834
  0.01083272 0.01658139]
 [0.1191096  0.17280697 0.18676217 0.20341631 0.17267432 0.11617595
  0.01039568 0.018659  ]
 [0.15401288 0.19921093 0.17824598 0.21088593 0.23281322 0.01880679
  0.00481193 0.00121233]
 [0.15114149 0.1992085  0.17840517 0.21222234 0.23300484 0.01973272
  0.00485107 0.00143386]
 [0.33233543 0.23668273 0.19678493 0.10855085 0.1221123  0.00154607
  0.0019877  0.        ]
 [0.14940006 0.20117671 0.17886335 0.2113231  0.23420304 0.01869665
  0.0052111  0.00112599]
 [0.11662746 0.17276934 0.18649792 0.20252499 0.17185824 0.11896735
  0.01068099 0.02007372]
 [0.1537799  0.19817186 0.17777237 0.21208466 0.23302627 0.01915254
  0

# [Q5](https://github.com/chauvinSimon/hmm_for_autonomous_driving/blob/master/README.md#q5) - Given a sequence of speed observations, what is the most likely underlying lane sequence?

0### Decoding
The **inferred optimal hidden states** can be obtained by calling [`decode()`](https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.MultinomialHMM.decode) method.

#Conversion to Dictionary from numpy array

In [64]:
def convert_to_dict(mat_val):
    rrr = {}
    for row in range(len(states)):
        key = states[row]
        dict = {}
        for col in range(len(states)):
            dict[states[col]] = mat_val[row][col]
        rrr[key] = dict
    return rrr

def convert_to_dict_speed(mat_val):
    rrr = {}
    for row in range(len(states)):
        key = states[row]
        dict = {}
        for col in range(len(observations)):
            dict[observations[col]] = mat_val[row][col]
        rrr[key] = dict
    return rrr

def convert_to_start_prob():
    dict = {}
    for i in range(len(states)):
        dict[states[i]] = initial_state_probability[i]
    return dict

#Viterbi Algorithm 

In [65]:
def Viterbit(obs, states, s_pro, t_pro, e_pro):
    path = { s:[] for s in states} # init path: path[s] represents the path ends with s
    curr_pro = {}
    for s in states:
        curr_pro[s] = s_pro[s]*e_pro[s][obs[0]]
    for i in range(1, len(obs)):
        last_pro = curr_pro
        curr_pro = {}
        for curr_state in states:
            max_pro, last_sta = max(((last_pro[last_state]*t_pro[last_state][curr_state]*e_pro[curr_state][obs[i]], last_state)
                                     for last_state in states))
            curr_pro[curr_state] = max_pro
            path[curr_state].append(last_sta)

    # find the final largest probability
    max_pro = -1
    max_path = None
    for s in states:
        path[s].append(s)
        if curr_pro[s] > max_pro:
            max_path = path[s]
            max_pro = curr_pro[s]
    # print '%s: %s'%(curr_pro[s], path[s]) # different path and their probability
    return max_path

In [66]:
obs_sequence = np.array([[2, 2, 2, 2, 2, 2, 2]]).T
# Find most likely state sequence corresponding to obs_sequence
logprob, state_sequence = discrete_model.decode(obs_sequence)

tp = convert_to_dict(transition_probability)
ep = convert_to_dict_speed(emission_probability)
ip = convert_to_start_prob()

# print(tp)

obs = ["low_speed", 'average_speed', "high_speed", "high_speed", 'average_speed', "high_speed", "high_speed", 'high_speed', "high_speed"]
print(Viterbit(obs, states, ip, tp, ep))
#
# print(state_sequence)
#
# # Log probability of the produced state sequence
# for o, s in zip(obs_sequence.T[0], state_sequence):
#     print("{} -> {}".format(states[int(s)], observations[int(o)]))
# print("\nprob = {}\nlog_prob = {}".format(np.exp(logprob), logprob))

['lane_0', 'lane_4', 'lane_3', 'lane_3', 'lane_4', 'lane_3', 'lane_3', 'lane_3', 'lane_3']
